# CAstro buses

In [ ]:
import pandas as pd
import numpy as np
from openpyxl import Workbook

## Carga de datos desde archivo CSV

In [24]:
empresas = pd.read_csv('../data/CASTRO-empresas.csv')
destinos = pd.read_csv('../data/CASTRO-destinos.csv')

In [25]:
# Establecer 'RUT' como índice en los dataframes existentes
for name in ['empresas', 'destinos']:
    df = globals().get(name)
    if df is None:
        print(f"{name} no está definido en el entorno.")
        continue
    if 'Rut' in df.columns:
        df.set_index('Rut', inplace=True)
    else:
        print(f"'Rut' no está en las columnas de {name}.")

In [ ]:
# cambia el nombre de 'Razon Social' a 'Operador' en el dataframe de empresas
empresas.rename(columns={'Razon Social': 'Operador'}, inplace=True)
empresas

,Operador,Empresa,Telefono,Correo
Rut,,,,
10122842-8,AURELIANO OJEDA GARAY,BUSES OJEDA,951759186.0,busesojeda@gmail.com
7086054-6,BALBINO ELADIO MONTIEL PAREDES,NaN,976162055.0,luis.montiel@live.cl
13409723-K,CARLOS SANCHEZ DIAZ,NaN,993091918.0,llamaca@gmail.com
13407746-8,CLAUDIO ALEJANDRO HUERQUE HUERQUE,NaN,958353794.0,claudio13huerque@gmail.com
10461960-6,CLAUDIO DEL RIO CALDERON,NaN,988468808.0,delrioclaudio21@gmail.com
...,...,...,...,...
77642229-0,TRANSPORTE LOS PELLINES LTDA. JORDAN,BUSES JORDAN,989055953.0,daniela.villanueva@busesjordan.cl
76407365-7,TRANSPORTES LOURDES MERKA SPA,COSTA BARRIA,996414915.0,costabarria@gmail.com
76687566-1,TRANSPORTES OSCAR SALDIVIA BARRIA EIRL,NaN,990470696.0,oscars_colt@hotmail.com


In [27]:
destinos

,Hora de Salida,Operador,Empresa,Destino,Andén,Observaciones
Rut,,,,,,
10122842-8,8:30,AURELIANO OJEDA GARAY,BUSES OJEDA,CUCAO,5,LUNES A DOMINGO
10122842-8,9:30,AURELIANO OJEDA GARAY,BUSES OJEDA,CUCAO,6,LUNES A DOMINGO
10122842-8,10:30,AURELIANO OJEDA GARAY,BUSES OJEDA,CUCAO,5,LUNES A DOMINGO
10122842-8,11:30,AURELIANO OJEDA GARAY,BUSES OJEDA,CUCAO,5,LUNES A DOMINGO
10122842-8,13:00,AURELIANO OJEDA GARAY,BUSES OJEDA,CUCAO,5,LUNES A DOMINGO
...,...,...,...,...,...,...
15291276-5,15:30,VALERIA DIAZ,NaN,ACHAO,1,LUNES A VIERNES
15291276-5,20:15,VALERIA DIAZ,NaN,ACHAO,1,LUNES A VIERNES
9468815-9,13:15,VICTOR CARTES CARTES,NaN,ACHAO,1,LUNES A VIERNES


In [28]:
# Crear 'sindestinos' con las empresas que no aparecen en 'destinos' (unión por índice 'Rut'). Usamos merge con indicador para identificar filas sin correspondencia en destinos
merged = empresas.merge(destinos, left_index=True, right_index=True, how='left', indicator=True)

# Filtrar solo las empresas sin destinos asociados
left_only = merged[merged['_merge'] == 'left_only']

sindestinos = empresas.loc[left_only.index].copy()

# Mostrar resultado
sindestinos

,Operador,Empresa,Telefono,Correo
Rut,,,,
76305760-7,EUROBUS (ETM),ETM,NaN,rbohle@busesetm.cl
7902557-7,FRANCISCO HERIBERTO DELGADO BARRIENTOS,NaN,940079402.0,fcodelgado56@gmail.com
6476541-6,JAIME ARNOLDO MUÑOZ CARDENAS,NaN,NaN,NaN
11118143-8,JOSE FRANCISCO CULUM ILNAO,NaN,990308328.0,fculun@gmail.com
4379279-2,JOSE HERACLIO ARRIAGADA NEIRA,NaN,982973227.0,NaN
9692568-9,JUAN ORLANDO DIAZ CARTES,NaN,999056884.0,marorl_bus@hotmail.com
14042296-7,NESTOR FABIAN LLAUQUEN LLEUQUEN,NaN,984625168.0,nestorllauquen@gmail.com
8688840-8,PEDRO ALFONSO PACHECO AGUILA,NaN,983100943.0,pedropacheco.aguila@gmail.com
76300596-8,QUEILEN BUS NORTE LTDA.,QUEILEN BUS,NaN,jorge@queilenbus.cl


In [29]:
# crear un exceel con el dataframe sindestinos
sindestinos.to_excel('../data/CASTRO-sindestinos.xlsx', index=True)

In [18]:
# RUTs que aparecen en 'destinos' pero no en 'empresas' -> crear 'sinempresa' con columnas 'Rut' y 'Operador'
mask = ~destinos.index.isin(empresas.index)
sinempresa = destinos.loc[mask, ['Operador']].reset_index().drop_duplicates(subset='Rut').reset_index(drop=True)
sinempresa

,Rut,Operador
0,NaN,CARLOS GOMEZ
